# Working with data from the HPF Spectrograph

by Michael Gully-Santiago & Jessica Luna

`muler` has new *beta support* for the [Habitable Zone Planet Finder Spectrograph](https://hpf.psu.edu/) (HPF).  In this tutorial we show the basic usage of how to read, process, and plot data from HPF.  We currently support data from either the Goldilocks pipeline or HPF Instrument Team pipeline.

In [ ]:
from muler.hpf import HPFSpectrum, HPFSpectrumList
import numpy as np
import glob

%config InlineBackend.figure_format='retina'

Here we have Goldilocks spectra:

In [ ]:
path = 'https://github.com/OttoStruve/muler_example_data/raw/main/HPF/01_A0V_standards/'
filename = 'Goldilocks_20210212T072837_v1.0_0037.spectra.fits'

We can easily read in HPF data for a specific spectral order:

In [ ]:
original_spectrum = HPFSpectrum(file=path+filename, order=15)

In [ ]:
original_spectrum.sky

The spectrum has physical units:

In [ ]:
original_spectrum.wavelength # "Angstroms"

In [ ]:
original_spectrum.flux.unit # "counts"

In [ ]:
original_spectrum.sky.flux.unit

We can normalize the spectrum, which divides the spectrum by the median value, rendering the flux units *dimensionless*

In [ ]:
spectrum = original_spectrum.normalize()
np.nanmedian(spectrum.flux)

We can effortlessly subtract the sky emission from the target fiber.

In [ ]:
sky_free_spectrum = spectrum.sky_subtract(method='vector')

Now we can normalize and overplot plot the observed spectrum, sky subtracted spectrum, and the sky emission itself:

In [ ]:
ax = spectrum.plot(label='Observed spectrum', color='k')
spectrum.sky.plot(ax=ax, label='Sky spectrum')
sky_free_spectrum.plot(ax=ax, label='Observed - Sky', lw=0.5)
ax.legend(ncol=3); ax.set_ylim(0.0, 1.5);

Nice! We have a sky subtracted spectrum!  Let's remove the instrumental response function from this sky subtracted spectrum.  The instrumental response is dominated by the characteristic concave-down shape.  This conspicuous parabola-like curve stems from an optical device called an [echelle grating](https://en.wikipedia.org/wiki/Echelle_grating) and its related "blaze function".  Accordingly the process of removing this shape is sometimes referred to by the names "de-blazing", "flattening", or "normalizing".  In this tutorial we will stick with the term "deblaze".

We first want to remove the `NaN` values at the edges, then apply the spline division:

In [ ]:
deblazed_spectrum = sky_free_spectrum.remove_nans().deblaze()

In [ ]:
ax = deblazed_spectrum.normalize().plot(label='Deblazed')
ax.axhline(1.0, linestyle='dashed', color='k')
ax.set_ylim(0.5, 1.2); ax.legend();

Great!  We have achieved our goal: sky subtracted and deblazed target spectrum ready for analysis.

You can see that the spline division is not perfect because some broad line wings can be mistaken as part of the blaze shape.  `muler` has experimental support for a different type of deblazing based on high-fidelity [flat-field](https://en.wikipedia.org/wiki/Flat-field_correction) spectra.  Those experimental techniques currently require ancillary calibration files that are not provided with our git repo.  Check back in for future updates!